In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install youtokentome

In [3]:

import youtokentome as yttm
from glob import glob

In [4]:
fs = [f for f in glob('../data/ru/**/**/*.txt')]

In [47]:
ls ../data/ru/publicism/

Tolstoy/


In [48]:
txts = []
fns = []
for fn in glob('../data/ru/**/**/*.txt'):
    with open(fn) as f:
        txt = f.read()
        txts.append(txt)
        fns.append(fn)

#traintxt = '\n'.join(txts)

In [59]:
import pandas as pd

df = pd.DataFrame({'text': txts, 'fn':fns})
df['fn'] = df.fn.apply(lambda x: x.split('/')[-2])

In [60]:
df['text'] = df.groupby(['fn'])['text'].transform(lambda x: '\n'.join(x))
df = df.drop_duplicates('text')

In [73]:
df['l'] = df.text.str.len()

In [74]:
df

,text,fn,l
0,"Из 7 миллиардов человек на Земле, примерно\n6 ...",tedtxt,6047793
10,Получил ваше интересное письмо и очень рад слу...,Tolstoy,10055085
36,Федор Михайлович Достоевский\nVI\n\nБОБОК\n\n ...,Dostoevsky,11221678
69,БРОЖЕНИЕ УМОВ\n(ИЗ ЛЕТОПИСИ ОДНОГО ГОРОДА)\nЗе...,Chekhov,1834329
146,\nТом первый\n\n\nГлава первая\n\nВ ворота гос...,Gogol,2289720
162,Михаил Лермонтов. Герой нашего времени\n\n\n\n...,Lermontov,323344
165,Максим Горький\nМАКАР ЧУДРА\nС моря дул влажны...,Gorky,4324024
240,\nИван Сергеевич Тургенев\nАся\n\n\nI\n\nМне б...,Turgenev,2345242
250,"Ни сны, ни явь\n\n Мы сидели на закате всем ...",Blok,40679
254,\nПовести покойного Ивана Петровича Белкина\n\...,Pushkin,932231


In [66]:
dfs = df.to_dict(orient='records')

In [71]:
for d in dfs:
    fn = d['fn']
    with open(f'../data/ru/{fn}.txt', 'w') as f:
        f.write(d['text'])

In [70]:
!pwd

/Users/ibragim/Documents/punctuation-restoration/src


In [12]:
import re
words = re.split(r'\W+', traintxt)

In [14]:
lowt = ' '.join(words)
lowt = lowt.lower()

In [17]:
train_data_path = "../data/train_data.txt"
model_path = "../yttme.model"


with open(train_data_path, "w") as fout:
    fout.write(traintxt)

# Generating random text
test_text = 'привет как твои дела?'

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=30000, model=model_path)

# Loading model
bpe = yttm.BPE(model=model_path)

# Two types of tokenization
print(bpe.encode([test_text], output_type=yttm.OutputType.ID))
print(bpe.encode([test_text], output_type=yttm.OutputType.SUBWORD))

[[7017, 365, 6622, 795]]
[['▁привет', '▁как', '▁твои', '▁дела']]


In [16]:
!pwd

/Users/ibragim/Documents/punctuation-restoration/src


In [18]:
import youtokentome as yttm


bpe = yttm.BPE("../yttme.model", n_threads=-1)

In [19]:
bpe.vocab_size()

30000

In [23]:
test_text = 'привет'
print(bpe.encode(test_text, output_type=yttm.OutputType.SUBWORD))

['▁привет']


In [5]:
from prep import Preprocessor
from glob import glob

pr = Preprocessor()

In [6]:
dfs = []
for fn in glob('../data/ru/**/**/*.txt'):
    df = pr.prep_file(fn)
    dfs.append(df)

In [7]:
import pandas as pd

aldf = pd.concat(dfs).reset_index(drop=True)

In [28]:
!pip install mmh3 -qq

In [30]:
from murhash import murmurhash

a = murmurhash('привет')

In [38]:
import numpy as np

np.array(a).shape

(256,)